# 2. Bag of Words(BoW)
> 단어의 등장 순서를 고려하지 않는 빈도수 기반의 단어 표현 방법

## 1) Bag of Words?
- 직역 : **'단어들의 가방'**
- BoW 만드는 과정
    - (1) 우선, 각 (서로 다른)단어에 고유한 정수 인덱스를 부여합니다.
    - (2) 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 만듭니다.

In [3]:
# 입력된 문서에 대해서 단어 집합(vocaburary)을 만들어 인덱스를 할당하고, BoW를 만드는 코드
from konlpy.tag import Okt
import re  
okt=Okt()  

token=re.sub("(\.)","","정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.")  
# 정규 표현식을 통해 온점을 제거하는 정제 작업입니다.  
token=okt.morphs(token)  
# OKT 형태소 분석기를 통해 토큰화 작업을 수행한 뒤에, token에다가 넣습니다.  

word2index={}  
bow=[]  
for voca in token:
    
    # token을 읽으면서, word2index에 없는 단어는 새로 추가
    if voca not in word2index.keys():  
        word2index[voca]=len(word2index)  
        bow.insert(len(word2index)-1,1)
        
    else:
        index=word2index.get(voca)  # 재등장하는 단어의 인덱스를 받아옵니다.
        bow[index]=bow[index]+1

print(word2index)
print(bow)

# 원한다면 한국어에서 불용어에 해당되는 조사들 또한 제거하여 더 정제된 BoW를 만들 수도 있습니다.

{'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
[1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


## 2) Bag of Words의 다른 예제들
- 중요한 것은 단어의 등장 빈도(단어의 순서, 즉 인덱스 순서는 상관없음)
### Example1(위에서 사용한 문장 그대로)
- ('발표': 0, '가': 1, '정부': 2, '하는': 3, '소비자': 4, '과': 5, '물가상승률': 6, '느끼는': 7, '은': 8, '다르다': 9)  
$\Longrightarrow$[1, 2, 1, 1, 1, 1, 2, 1, 1, 1]  
- 위의 BoW는 단지 단어들의 인덱스만 바뀌었을 뿐이며, 개념적으로는 여전히 앞서 만든 BoW와 동일한 BoW로 취급할 수 있음

### Example2 : 소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.  
- ('소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가상승률': 9, '느낀다': 10)  
$\Longrightarrow$[1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]

### Example3 : 정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다. 소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.  
- ('정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9, '는': 10, '주로': 11, '소비': 12, '상품': 13, '을': 14, '기준': 15, '으로': 16, '느낀다': 17)   
$\Longrightarrow$[1, 2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]  

Example3의 단어 집합은 Example1과 Example2의 단어들을 모두 포함하고 있는 것들을 볼 수 있습니다. BoW는 종종 여러 문서의 단어 집합을 합친 뒤에, 해당 단어 집합에 대한 각 문서의 BoW를 구하기도 합니다. 가령, Example3에 대한 단어 집합을 기준으로 Example1, Example2의 BoW를 만든다고 한다면 결과는 아래와 같습니다.

- Example3 단어 집합에 대한 Example1 BoW : [1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
- Example3 단어 집합에 대한 Example2 BoW : [0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 2, 1, 1, 1]  

문서3 단어 집합에서 물가상승률이라는 단어는 인덱스가 4에 해당됩니다. 물가상승률이라는 단어는 문서1에서는 2회 등장하며, 문서2에서는 1회 등장하였기 때문에 두 BoW의 인덱스 4의 값은 각각 2와 1이 되는 것을 볼 수 있습니다.

## 3) CountVectorizer 클래스로 BoW 만들기
- CountVectorizer : Scikit-learn에서 지원하는 클래스로 단어 빈도 count하여 vector로 만들어줌

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = ['you know I want your love. because I love you.']
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray()) # 코퍼스로부터 각 단어의 빈도 수를 기록한다.
print(vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

[[1 1 2 1 2 1]]
{'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


- CountVectorizer는 **단지 띄어쓰기만을 기준으로 단어를 자르는 낮은 수준의 토큰화를 진행하고 BoW를 만든다.**
    - <ins>영어</ins>는 띄어쓰기만으로 토큰화가 수행되서 문제 없음
    - <ins>한국어</ins>는 조사 등의 이유로 제대로 BoW가 만들어지지 않음
        - '물가상승률'을 위에서는 인식 못함...
        - '물가상승률과'와 '물가상승률은' 으로 조사를 포함해서 하나의 단어로 판단하기 때문에 서로 다른 두 단어로 인식


## 4) 불용어를 제거한 BoW 만들기

- BoW를 사용한다는 것은 그 문서에서 각 단어가 얼마나 자주 등장했는지를 보겠다는 것
- 각 단어에 대한 빈도수를 수치화 하겠다는 것은 결국 **텍스트 내에서 어떤 단어들이 중요한지를 보고싶다는 의미**를 함축  
$\Longrightarrow$ BoW를 만들때 **불용어를 제거하는 일**은 자연어 처리의 정확도를 높이기 위해서 선택할 수 있는 전처리 기법

### 사용자가 직접 정의한 불용어 사용

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

text=["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words=["the", "a", "an", "is", "not"])
print(vect.fit_transform(text).toarray()) 
print(vect.vocabulary_)

[[1 1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


### CountVectorizer에서 제공하는 자체 불용어 사용

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

text=["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words="english")
print(vect.fit_transform(text).toarray())
print(vect.vocabulary_)

[[1 1 1]]
{'family': 0, 'important': 1, 'thing': 2}


### NLTK에서 지원하는 불용어 사용

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

text=["Family is not an important thing. It's everything."]
sw = stopwords.words("english")
vect = CountVectorizer(stop_words =sw)
print(vect.fit_transform(text).toarray()) 
print(vect.vocabulary_)

[[1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 3, 'everything': 0}
